In [1]:
# in order to use very bit of submit file size limit,
# as well as lower score_thr to get more bonus

# submit file limit : 20M

In [2]:
import mmcv
import pandas as pd
import numpy as np


from tqdm import tqdm
from datetime import datetime
import os

In [3]:
def iter_results(pred):
    for idx, bboxes in enumerate(pred): # enumerate on classes, only has one in this competition
        for x_min, y_min, x_max,y_max, p in bboxes:
            x_min  =int(x_min)
            y_min = int(y_min)
            x_max = int(x_max)
            y_max = int(y_max)
            yield x_min, y_min, x_max, y_max, p 

In [4]:
img_info  =mmcv.load('../data/icartoonface/dtest.pkl')
pred = mmcv.load('test.pkl')

assert len(pred) == len(img_info)

submit = []
for _img, _pred in tqdm(zip(img_info, pred), total=len(pred)):
    for x_min, y_min, x_max, y_max , p in iter_results(_pred):
        submit.append([
            _img['filename'].split('/')[-1], x_min, y_min, x_max, y_max, 'face', p
        ])
        
submit = pd.DataFrame(submit)

100%|██████████| 10000/10000 [00:03<00:00, 2524.64it/s]


In [5]:
def compress(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



In [6]:
# compress DataFrame
submit = compress(submit)
# submit.info(memory_usage='deep')

Mem. usage decreased to 24.80 Mb (53.6% reduction)


In [7]:
# make submission first to get current size for reference
subname = 'submit.csv'
pd.DataFrame(submit).to_csv(subname,
                           index=False, header=False)

In [8]:
def top_select(pred, comp):
    ps = []
    for _pred in pred:
        for bbox in _pred[0]:
            ps.append(bbox[-1])
    
    ps = np.array(ps)
    ps_idx = np.argsort(-ps)[:int(comp * len(ps))]
    thr = ps[ps_idx][-1]
    print(thr)
    
    n_start = len(ps)
    n_end =len(ps_idx)
    print(n_start, n_end)
    
    comp_pred = []
    for _pred in pred:
        _pred = _pred[0] # only class 1
        comp_pred.append([_pred[_pred[:,-1] > thr]])
    
    return np.array(comp_pred)

In [9]:
!ls -hl submit.csv

-rw-r--r-- 1 qixin.xqx users 66M Jun  3 20:45 submit.csv


In [10]:
subsize = 66


In [11]:
if subsize > 20:
    pred = top_select(pred, 20 / subsize)

0.000464619
1000000 303030


In [12]:
# renew submit file
submit = []
for _img, _pred in tqdm(zip(img_info, pred), total=len(pred)):
    for x_min, y_min, x_max, y_max , p in iter_results(_pred):
        submit.append([
            _img['filename'].split('/')[-1], x_min, y_min, x_max, y_max, 'face', p
        ])
        
submit = pd.DataFrame(submit)

100%|██████████| 10000/10000 [00:01<00:00, 9931.72it/s]


In [13]:
# compress DataFrame
submit = compress(submit)
submit.info(memory_usage='deep')

subname = datetime.now().strftime('submit%m%dT%H%M') + '.csv'
pd.DataFrame(submit).to_csv(subname,
                           index=False, header=False)

Mem. usage decreased to  7.50 Mb (53.6% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302663 entries, 0 to 302662
Data columns (total 7 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       302663 non-null  object 
 1   1       302663 non-null  int16  
 2   2       302663 non-null  int16  
 3   3       302663 non-null  int16  
 4   4       302663 non-null  int16  
 5   5       302663 non-null  object 
 6   6       302663 non-null  float16
dtypes: float16(1), int16(4), object(2)
memory usage: 47.9 MB
